# **Lab 3: approximation**
**Jesper Lidbaum**

# **Abstract**

In this report, we will implement an L2 projection on a one-dimensional real function onto a piecewise linear function space. The error was $< 0.00085$ for the real function $f(x) = sin(x)$ projected onto the piecewise linear space with 100 nodes. And the order of convergence was estimated to be quadratic.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [17]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2023 Jesper Lidbaum (jlidbaum@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need 
additional modules, this is where you add them. 

In [18]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

The problem we investigate in this report is projecting a real one-dimensional function that we assume is continuous, onto a function space of piecewise linear functions. This is done with the base hat functions $\phi_i$ for each node. $\phi_i$ takes on the form of a linear function that goes from 0 to 1 in the interval preceding the node and a linear function from 1 to 0 in the next interval. The special case is in the first and last node where the function is cut in half. 

# **Method**

First, we define the linear shape functions for a sub-interval they take on an increasing or decreasing form depending on the parameter $p$. They are from example 9.6 in the course book. The x_vector is the vector of all nodes in the interval. x is the parameter for the function. k is the node in the middle of the interval.

In [19]:
def lambda_f(k, p, x_vector, x):
    if(p == 0):
        return (x_vector[k] - x)/(x_vector[k] - x_vector[k-1])
    else:
        return (x - x_vector[k - 1])/(x_vector[k] - x_vector[k-1])

Next, we define the base functions for the interval. The base functions are from example 9.6 in the course book.

In [20]:
def phi_f(i, x, x_vector):
    if(i == 0):
        if(x < x_vector[0] or x > x_vector[1]):
            return 0
        return lambda_f(1, 0, x_vector, x)
    elif(i == len(x_vector) - 1):
        if(x < x_vector[i-1] or x > x_vector[i]):
            return 0
        return lambda_f(i, 1, x_vector, x)
    else:
        if(x < x_vector[i-1] or x > x_vector[i+1]):
            return 0
        elif(x < x_vector[i]):
            return lambda_f(i, 1, x_vector, x)
        else:
            return lambda_f(i+1, 0, x_vector, x)

Next, we define the inner product for the basis functions $(\phi_i, \phi_j)$. By figure 9.6 in the course book, we know that it is zero except when $i=j$, $i=j+1$, and $i=j-1$. The integral can be solved generically for the three cases and returned directly. 

In [21]:
def phi_dot(i,j, x_vector):
    if(i == 0 and j == 0):
        return (x_vector[i+1] - x_vector[i]) / 3
    if(i == len(x_vector) - 1 and j == len(x_vector) - 1):
        return (x_vector[i] - x_vector[i-1]) / 3
    if(i==j):
        return (x_vector[i] - x_vector[i-1]) / 3 + (x_vector[i+1] - x_vector[i]) / 3
    if(i==j-1):
        return (x_vector[j] - x_vector[i]) / 6
    if(i==j+1):
        return (x_vector[i] - x_vector[j]) / 6
    return 0


Next, we need the value for $(f, \phi_i) = \int_{I_i}f(x)\phi_i(x)dx$ where f is the function we want to project. Again we have the special cases for the first and last node. The integral is solved with the trapezoidal rule.

In [22]:
def phi_dot_f(i, f, x_vector):
    if(i == 0):
        integrand = lambda x: ((x_vector[1] - x) / (x_vector[1] - x_vector[0])) * f(x)
        space = np.linspace(x_vector[0], x_vector[1], 100)
        return np.trapz(integrand(space), space, dx = 0.01)
    elif(i == len(x_vector) - 1):
        integrand = lambda x: ((x - x_vector[i-1]) / (x_vector[i] - x_vector[i-1])) * f(x)
        space = np.linspace(x_vector[i-1], x_vector[i], 100)
        return np.trapz(integrand(space), space, dx = 0.01)

    integrand = lambda x: ((x - x_vector[i-1]) / (x_vector[i] - x_vector[i-1])) * f(x)
    space = np.linspace(x_vector[i-1], x_vector[i], 100)
    left = np.trapz(integrand(space), space, dx = 0.01)
    integrand = lambda x: ((x_vector[i+1] - x) / (x_vector[i+1] - x_vector[i])) * f(x)
    space = np.linspace(x_vector[i], x_vector[i+1], 100)
    right = np.trapz(integrand(space), space, dx = 0.01)
    return left + right

Lastly, we use equations 9.18 and 9.19 from the course book and use the solution vector $\alpha$ to perform the projection. 

In [23]:
def l2_projection(alpha, x, x_vector, i):
    if(i == 0):
        return alpha[i] * phi_f(i, x, x_vector) + alpha[i+1] * phi_f(i+1, x, x_vector)
    elif(i == len(x_vector) - 1):
        return alpha[i-1] * phi_f(i-1, x, x_vector) + alpha[i] * phi_f(i, x, x_vector)
    return alpha[i-1] * phi_f(i-1, x, x_vector) + alpha[i] * phi_f(i, x, x_vector) + alpha[i+1] * phi_f(i+1, x, x_vector)


def l2_projection_over_points(x_vector, f):
    A = np.zeros((len(x_vector), len(x_vector)))
    for i in range(len(x_vector)):
        for j in range(len(x_vector)):
            A[i][j] = phi_dot(i, j, x_vector)
    b = np.zeros(len(x_vector))
    for i in range(len(x_vector)):
        b[i] = phi_dot_f(i, f, x_vector)

    alpha = np.linalg.solve(A, b)

    ret = np.zeros(len(x_vector))

    for i in range(len(x_vector)):
        ret[i] = l2_projection(alpha, x_vector[i], x_vector, i)

    return ret

# **Results**

Now we use the maximal error from the real function in the projected sequence to approximate the order of convergence. We also get the maximum error for sequences with 99 and 100 nodes respectively. We can see that the rate of convergence seems to be quadratic.

In [24]:
x_vector = np.linspace(0.0, 10.0, 99)
f = lambda x: np.sin(x)
real_values = f(x_vector)
approx_values = l2_projection_over_points(x_vector, f)

x_vector2 = np.linspace(0.0, 10.0, 100)
real_values2 = f(x_vector2)
approx_values2 = l2_projection_over_points(x_vector2, f)

error1 = np.abs(real_values - approx_values)
error2 = np.abs(real_values2 - approx_values2)

print("Error1: ", np.max(error1))
print("Error2: ", np.max(error2))

print("Estimated rate of convergence: ", np.log(np.max(error2)/np.max(error1))/np.log(99/100))

Error1:  0.0008680791260962106
Error2:  0.0008503673703786907
Estimated rate of convergence:  2.0511168328048033


# **Discussion**

The interesting part about this lab was that I had to connect the dots in the mathematical formulas from the course book and lectures to create code. I had to in a way convert the math notation to how I think about it as a programmer. There is quite a difference. But when I got it it was pretty simple to implement.

The implication that this method has is pretty clear on functions that are either hard to compute. But I see how we could use these approximations to solve differential equations later. 
